In [1]:
# force the notebook to auto reload external python modules | useful for development
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import sys

sys.path.append("..\\")
from src.data import MyDataset, AudioTripletDataset, LandmarkTripletDataset, AudioLandmarkTripletDataset
from src.features import TripletGenerator
# from src.models import main

import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader

random_seed = 42

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [4]:
# paths
raw_data_path = "..\\data\\raw\\"
processed_data_path = "..\\data\\processed\\"

---

In [5]:
my_dataset = MyDataset(location=raw_data_path)

In [ ]:
# create & pickle dataset | RUN THIS CELL ONLY ONCE TO CREATE THE DATASET
my_dataset.create_dataset()
my_dataset.save_dataset(processed_data_path)

In [ ]:
# load dataset | RUN THIS CELL TO LOAD THE SAVED DATASET
my_dataset.load_dataset(processed_data_path)
sample_size = len(my_dataset)
print(f"Loaded dataset with {sample_size} samples.")

In [ ]:
# train-valid-test split
# to maintain independence between train and test sets, split the dataset before creating the triplets

train_size = int(0.6 * sample_size)
val_size = int(0.2 * sample_size)
test_size = sample_size - train_size - val_size

train_set, val_set, test_set = random_split(my_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(random_seed))

train_size, valid_size, test_size = len(train_set), len(val_set), len(test_set)
print(f"Train dataset size: {train_size} \nValid dataset size: {valid_size} \nTest dataset size: {test_size}")

In [ ]:
# For Stratified Split

from sklearn.model_selection import train_test_split

# Assuming my_dataset is an instance of a Dataset class
sample_size = len(my_dataset)
targets = [my_dataset[i][1] for i in range(sample_size)]  # Assuming the second element is the label

# Splitting the data into train, validation, and test with stratification
train_idx, temp_idx, train_targets, temp_targets = train_test_split(
    range(sample_size), targets, test_size=0.2, stratify=targets, random_state=random_seed
)

val_idx, test_idx, val_targets, test_targets = train_test_split(
    temp_idx, temp_targets, test_size=0.2, stratify=temp_targets, random_state=random_seed
)

# Creating subsets
train_set = torch.utils.data.Subset(my_dataset, train_idx)
val_set = torch.utils.data.Subset(my_dataset, val_idx)
test_set = torch.utils.data.Subset(my_dataset, test_idx)

# Sizes of the datasets
train_size, valid_size, test_size = len(train_set), len(val_set), len(test_set)
print(f"Train dataset size: {train_size} \nValid dataset size: {valid_size} \nTest dataset size: {test_size}")


In [ ]:
# CREATE & PICKLE triplets of indicies | RUN THIS CELL ONLY ONCE TO CREATE THE TRIPLETS
# ensure diversity of triplets by setting num_triplets to a large number
# essentially 10 to 20 triplets per sample

num_triplets = 20

# TRAIN
train_triplets = TripletGenerator(train_set, num_triplets=train_size*num_triplets, prefix="train")
train_triplets.save_triplets(processed_data_path)

# VALID
valid_triplets = TripletGenerator(val_set, num_triplets=valid_size*num_triplets, prefix="valid")
valid_triplets.save_triplets(processed_data_path)

# TEST
test_triplets = TripletGenerator(test_set, num_triplets=test_size*num_triplets, prefix="test")
test_triplets.save_triplets(processed_data_path)

print(f"Created and saved: \
      \n {len(train_triplets.triplets)} train triplets \
      \n {len(valid_triplets.triplets)} valid triplets \
      \n {len(test_triplets.triplets)} test triplets"
      )

---

In [ ]:
# LOAD triplets | RUN THIS CELL TO LOAD THE SAVED TRIPLETS

# TRAIN
train_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="train")
train_AL_triplets_dataset = AudioLandmarkTripletDataset(my_dataset.data, train_triplets.triplets)
train_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, train_triplets.triplets)
train_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, train_triplets.triplets)

# VALID
valid_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="valid")
valid_AL_triplets_dataset = AudioLandmarkTripletDataset(my_dataset.data, valid_triplets.triplets)
valid_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, valid_triplets.triplets)
valid_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, valid_triplets.triplets)

# TEST
test_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="test")
test_AL_triplets_dataset = AudioLandmarkTripletDataset(my_dataset.data, test_triplets.triplets)
test_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, test_triplets.triplets)
test_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, test_triplets.triplets)

print(f"Loaded: \
      \n {len(train_triplets.triplets)} train triplets \
      \n {len(valid_triplets.triplets)} valid triplets \
      \n {len(test_triplets.triplets)} test triplets"
      )

In [8]:
# imports
from src.config import SystemConfig, TrainingConfig
from src.logging import setup_log_directory
from src.models import main
from src.visualization import plot_loss_accuracy

from torch.utils.tensorboard import SummaryWriter

In [9]:
def train_model(
        train_dataset, 
        valid_dataset, 
        model,
    ):
    
    # create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=TrainingConfig.batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=TrainingConfig.batch_size, shuffle=True)

    # train model
    print(model)

    training_config = TrainingConfig()

    # Model checkpoint log dir setup.
    training_config, current_version_name = setup_log_directory(training_config)

    # Tensorboard log dir setup.
    summary_writer = SummaryWriter(training_config.log_dir)

    # Train and Validate
    train_loss, train_acc, val_loss, val_acc = main(
        model,
        (train_loader, valid_loader),
        summary_writer=summary_writer,
        scheduler=None,
        system_config=SystemConfig(),
        training_config=training_config,
        data_augmentation=False,
    )

    # plot loss and accuracy
    plot_loss_accuracy(
        train_loss=[train_loss],
        val_loss=[val_loss],
        train_acc=[train_acc],
        val_acc=[val_acc],
        colors=["blue"],
        loss_legend_loc="upper center",
        acc_legend_loc="upper left",
    )
    

### Audio model

In [13]:
# importing models from src.models
from src.models import LSTM, LSTM_II, CNN

In [ ]:
audio_data_shape=(250, 400)
model = LSTM_II(input_shape=audio_data_shape)
train_model(
    train_audio_triplet_dataset, 
    valid_audio_triplet_dataset, 
    model,
)

### Landmark Model

In [ ]:
from src.models import STGCN

In [ ]:
model = STGCN(num_nodes=20, in_channels=3, out_channels=64, num_frames=250)
train_model(
    train_landmark_triplet_dataset, 
    valid_landmark_triplet_dataset, 
    model,
)

### Combined model

### Testing

In [ ]:
# code for testing goes here - TBD